In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# 第一个agent
- 会做数学题
- 不知道答案的时候可以搜索
<hr>

In [2]:
#定义llm
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


搭建工具
- serpai是一个聚合搜索引擎，需要安装谷歌搜索包以及申请账号 https://serpapi.com/manage-api-key
- llm-math是一个封装好的数学计算链

In [3]:
! pip install google-search-results

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import os 
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"

In [5]:
from langchain.agents import load_tools

tools = load_tools(["serpapi","llm-math"], llm=llm)

定义agent,使用小样本增强生成类型

In [6]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,#这里有不同的类型
    verbose=True,#是否打印日志
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
agent.run("请问现任的美国总统是谁？他的年龄的平方是多少?")

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I need to find out who the current president of the United States is and then calculate the square of his age.
Action: Search
Action Input: current president of the United States
Observation: Joe Biden
Thought:Now I need to find out Joe Biden's age and calculate the square of it.
Action: Search
Action Input: Joe Biden age
Observation: 81 years
Thought:I will now calculate the square of Joe Biden's age.
Action: Calculator
Action Input: 81 * 81
Observation: Answer: 6561
Thought:I now know the final answer.
Final Answer: Joe Biden, 81 years old, square of his age is 6561.

> Finished chain.


'Joe Biden, 81 years old, square of his age is 6561.'